In [1]:
from pandas.io.json import json_normalize
import pandas as pd

In [2]:
pd.__version__

'0.19.0'

In [3]:
df_checkin = pd.read_json('../../dataset/academic/yelp_academic_dataset_checkin.json', lines=True)

In [4]:
df_checkin.head()

,business_id,checkin_info,type
0,cE27W9VPgO88Qxe4ol6y_g,"{'9-5': 1, '7-5': 1, '14-6': 1, '18-4': 1, '17...",checkin
1,mVHrayjG3uZ_RLHkLj-AMg,"{'11-5': 1, '6-2': 1, '10-5': 1, '15-4': 2, '1...",checkin
2,mYSpR_SLPgUVymYOvTQd_Q,"{'17-3': 1, '13-5': 1, '11-4': 1, '9-3': 1, '1...",checkin
3,KayYbHCt-RkbGcPdGOThNg,"{'11-1': 2, '21-2': 1, '14-3': 2, '17-3': 2, '...",checkin
4,b12U9TFESStdy7CsTtcOeg,"{'5-1': 2, '8-2': 1, '13-2': 1, '8-6': 1, '5-5...",checkin


In [5]:
df_checkin_info_3 = json_normalize(df_checkin['checkin_info'][3])
df_checkin_info_3

,10-3,10-4,10-5,11-0,11-1,11-2,12-2,12-4,13-0,13-2,...,21-2,21-4,21-5,22-5,23-1,8-1,8-3,8-4,9-1,9-4
0,1,2,1,2,2,2,2,3,1,2,...,1,1,2,1,1,2,1,1,1,5


In [6]:
df_checkin_info_3['17-3']

0    2
Name: 17-3, dtype: int64

## We start basic analysis from here.

In [7]:
import sys
import operator

pd.set_option('display.max_colwidth', sys.maxsize)

chunk_size = 100000
reader = pd.read_table('../../dataset/academic/yelp_academic_dataset_review.json', header=None, chunksize=chunk_size)

count_of_stars = {}
num_reviews_of_business = {}
active_user_of_business = {}
votes_of_business = {}

chunk_index = 0
for chunks in reader:
    
    start_index = chunk_size * chunk_index
    for index in range(start_index, start_index + len(chunks)):
        data_json = pd.read_json(chunks[0][index], typ='series')

        stars = data_json['stars']
        if (stars not in count_of_stars):
            count_of_stars[stars] = 1
        else: 
            count_of_stars[stars] = count_of_stars[stars] + 1

        business_id = data_json['business_id']
        if (business_id not in num_reviews_of_business):
            num_reviews_of_business[business_id] = 1
        else: 
            num_reviews_of_business[business_id] = num_reviews_of_business[business_id] + 1

        user_id = data_json['user_id']
        user_business_key = business_id + '_' + user_id
        if (user_business_key not in active_user_of_business):
            active_user_of_business[user_business_key] = 1
        else: 
            active_user_of_business[user_business_key] = active_user_of_business[user_business_key] + 1
            
        votes_json = data_json['votes']
        num_votes = votes_json['funny'] + votes_json['useful'] + votes_json['cool']
        if (business_id not in votes_of_business):
            votes_of_business[business_id] = num_votes
        else: 
            votes_of_business[business_id] = votes_of_business[business_id] + num_votes
        
    
    chunk_index += 1

#     if (chunk_index > 2):
#         break

print("\nTotal count of each different stars:")
print(count_of_stars)

print("\nSort business by number of reviews:")
sorted_num_reviews_of_business = sorted(num_reviews_of_business.items(), key=operator.itemgetter(1), reverse=True)
print(sorted_num_reviews_of_business[:10])

print("\nSort users by number of reviews on the same business:")
sorted_active_user_of_business = sorted(active_user_of_business.items(), key=operator.itemgetter(1), reverse=True)
print(sorted_active_user_of_business[:10])

print("\nSort business by number of votes from reviews:")
sorted_votes_of_business = sorted(votes_of_business.items(), key=operator.itemgetter(1), reverse=True)
print(sorted_votes_of_business[:10])


Total count of each different stars:
{1: 331786, 2: 224334, 3: 321700, 4: 674636, 5: 1132610}

Sort business by number of reviews:
[('4bEjOyTaDG24SY5TxsaUNQ', 5558), ('sIyHTizqAiGu12XMLX3N3g', 4531), ('zt1TpTuJ6y9n551sw9TaEg', 4333), ('aGbjLWzcrnEx2ZmMCFm3EA', 4149), ('2e2e7WgqU1BnpxmQL5jbfw', 4103), ('iUPJmJvHy9fVfRxsuwwdLQ', 3584), ('Xhg93cMdemu5pAMkDoEdtQ', 3330), ('QbmcCE_cLq4WO8ZMKImaLw', 3202), ('YNQgak-ZLtYJQxlDwN-qIg', 3107), ('lliksv-tglfUz1T3B3vgvA', 2911)]

Sort users by number of reviews on the same business:
[('T1jYZFB_7cqdhuvzpxfFWQ_q200TCFOheJfS_MvvwMfJw', 31), ('6LoNRotewF81fxka6L_8Jw_q200TCFOheJfS_MvvwMfJw', 19), ('yumgsa9m02CACzq0dHZP7Q_q200TCFOheJfS_MvvwMfJw', 18), ('8aPTQeDYOMNEu907WfflCQ__Q4Ks28-Jks-TjFT-YXw_A', 18), ('DzL4cUt5AVYRzzT7DHOgxg_GfJBijGnGRIbXRkNMzS6Eg', 16), ('6O9iAmAv7vUl-Eeqb9bmpw_C2RiYrCAdvj7Sh0p6I8G7Q', 16), ('iaEqryy7OwAOAQkLWDBquQ_kG8jkVFgOAOq6Ht6RvAjEQ', 15), ('RZTcKugP9dGWFkO4RpbVEQ_FbdB9KXyuvT8rC_zmqRSuQ', 15), ('zsAlOIoPCdVZO2kVAb5YJw_pk6ci-